In [66]:
#USPS preprocessing

import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
# time to import the data!
df_usps_train = pd.read_csv('zip.train',delim_whitespace=True,header=None)
df_usps_test = pd.read_csv('zip.test',delim_whitespace=True,header=None)

In [36]:
#see the data a bit

print(df_usps_train.head())
print(df_usps_train.shape)
print(df_usps_test.shape)

   0    1    2    3      4      5      6      7      8      9    ...    247  \
0  6.0 -1.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -0.631  0.862  ...  0.304   
1  5.0 -1.0 -1.0 -1.0 -0.813 -0.671 -0.809 -0.887 -0.671 -0.853  ... -0.671   
2  4.0 -1.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000  ... -1.000   
3  7.0 -1.0 -1.0 -1.0 -1.000 -1.000 -0.273  0.684  0.960  0.450  ... -0.318   
4  3.0 -1.0 -1.0 -1.0 -1.000 -1.000 -0.928 -0.204  0.751  0.466  ...  0.466   

     248    249    250    251    252    253    254    255  256  
0  0.823  1.000  0.482 -0.474 -0.991 -1.000 -1.000 -1.000 -1.0  
1 -0.671 -0.033  0.761  0.762  0.126 -0.095 -0.671 -0.828 -1.0  
2 -1.000 -1.000 -0.109  1.000 -0.179 -1.000 -1.000 -1.000 -1.0  
3  1.000  0.536 -0.987 -1.000 -1.000 -1.000 -1.000 -1.000 -1.0  
4  0.639  1.000  1.000  0.791  0.439 -0.199 -0.883 -1.000 -1.0  

[5 rows x 257 columns]
(7291, 257)
(2007, 257)


In [39]:
#train on USPS
#label data as -1 iff digit <=4, else 1
df_usps_modified = df_usps_train.copy()
for i in range(df_usps_modified.shape[0]):
    if df_usps_modified.loc[i,0] <=4:
        df_usps_modified.loc[i,0] = -1
    else:
        df_usps_modified.loc[i,0] = 1


df_usps_modified_test = df_usps_test.copy()
for i in range(df_usps_modified_test.shape[0]):
    if df_usps_modified_test.loc[i,0] <=4:
        df_usps_modified_test.loc[i,0] = -1
    else:
        df_usps_modified_test.loc[i,0] = 1
#print(df_usps_modified)
#print(df_usps_modified_test)
print(len(df_usps_modified[df_usps_modified.loc[:,0]==1].index))

3051


In [64]:
'''for i in range(1000):
    seed_value=i
    #sample 2000 from them
    df_usps_sample = df_usps_modified.sample(2000,random_state=seed_value)

    #print(df_usps_sample)
    size = len(df_usps_sample[df_usps_sample.loc[:,0]==1].index)
    #print(size)
    if (np.abs(size-1000)<115):
        print(seed_value)'''
        
seed_value=1
df_usps_sample_A = df_usps_modified.loc[df_usps_modified.loc[:,0]==1].sample(1000,random_state=seed_value)
df_usps_sample_B = df_usps_modified.loc[df_usps_modified.loc[:,0]==-1].sample(1000,random_state=seed_value)
df_usps_sample = pd.concat([df_usps_sample_A,df_usps_sample_B])
print(len(df_usps_sample[df_usps_sample.loc[:,0]==1].index))
print(df_usps_sample)

1000
      0    1    2      3      4      5      6      7      8      9    ...  \
15    1.0 -1.0 -1.0 -1.000 -0.929  0.351  0.798  0.806  0.114  0.015  ...   
6317  1.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -1.000 -0.973  0.612  ...   
6848  1.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 -0.531  ...   
1510  1.0 -1.0 -1.0 -0.695  0.805  0.768 -0.826 -1.000 -1.000 -1.000  ...   
2057  1.0 -1.0 -1.0 -1.000 -1.000 -0.812  0.300  0.952  1.000  0.956  ...   
...   ...  ...  ...    ...    ...    ...    ...    ...    ...    ...  ...   
2978 -1.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -0.872 -0.010  0.592  ...   
7126 -1.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000  ...   
3713 -1.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -0.943 -0.017  1.000  ...   
1624 -1.0 -1.0 -1.0 -1.000 -1.000 -1.000 -0.999  0.384  0.975 -0.005  ...   
4182 -1.0 -1.0 -1.0 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 -0.991  ...   

        247    248    249    250    251    252  253  254  255  256  
1

In [65]:
#method of partitioning data fram in 50*40
def partition_dataframe(df):
    result=[]
    df_to_sample_from = df.copy()
    for i in range(0,50):
        train = df_to_sample_from.sample(40,random_state=seed_value)
        remaining = df_to_sample_from.drop(train.index)
        df_to_sample_from = remaining
        result.append(train)
    return result
#store the sample
partition = partition_dataframe(df_usps_sample)
dataframe_file = [partition,df_usps_sample, df_usps_modified_test]
with open('usbs_databases', 'wb') as fp:
    pickle.dump(dataframe_file, fp)